In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_9\Log\training.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_9\Data\Preprosessed\Preprosessed_Simple.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.head()

,Unnamed: 0,index,dateCrawled,name,price,vehicleType,yearOfRegistration,powerPS,model,kilometer,...,seller_gewerblich,seller_privat,offerType_Angebot,offerType_Gesuch,abtest_control,abtest_test,gearbox_automatik,gearbox_manuell,notRepairedDamage_ja,notRepairedDamage_nein
0,0,0.000000,0.586679,0.339173,2.235174e-07,0.857143,0.110346,0.00000,0.472,150000,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,1,0.000003,0.585720,0.017608,8.521602e-06,0.428571,0.112346,0.00950,0.472,125000,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,2,0.000005,0.272956,0.387805,4.563481e-06,1.000000,0.111568,0.00815,0.476,125000,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,3,0.000008,0.378625,0.327975,6.984919e-07,0.571429,0.111235,0.00375,0.472,150000,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,4,0.000011,0.824117,0.730069,1.676381e-06,0.571429,0.112012,0.00345,0.412,90000,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


# Models Training Process 

# RandomForestRegressor TRAINING 

In [8]:
# Regression Models Comparison for target 'kilometer'
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Regression models
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb

# --------------------------
# 1️⃣ X va y ajratish
# --------------------------
target_col = 'kilometer'
X = df.drop(target_col, axis=1)
y = df[target_col]

# Train / Validation / Test (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# --------------------------
# 2️⃣ Modellarni yaratish
# --------------------------
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=5),
    "SVR": SVR(),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# --------------------------
# 3️⃣ Model trening va baholash
# --------------------------
results = []

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    # Predict
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    
    # Baholash
    r2_val = r2_score(y_val, y_pred_val)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_test = r2_score(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    
    results.append({
        "Model": name,
        "R2_Validation": r2_val,
        "MSE_Validation": mse_val,
        "R2_Test": r2_test,
        "MSE_Test": mse_test
    })

# --------------------------
# 4️⃣ Natijalarni jadval shaklida chiqarish
# --------------------------
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="R2_Validation", ascending=False))


ModuleNotFoundError: No module named 'xgboost'

# DecisionTreeRegressor TRAINING 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Model
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Baholash
y_pred_dt = dt.predict(X_val)
print("📊 Decision Tree Validation R2:", r2_score(y_val, y_pred_dt))
print("📉 Validation MSE:", mean_squared_error(y_val, y_pred_dt))

# Cros Validation Tekshirish Natijalari 

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Model
dt = DecisionTreeRegressor(random_state=42)

# R2 Score bo‘yicha 5-fold cross-validation
r2_scores = cross_val_score(dt, X, y, cv=5, scoring='r2')
print("📊 R2 (5-fold):", r2_scores)
print("📈 R2 O'rtacha:", np.mean(r2_scores))

# Cross-val prediction orqali MSE hisoblash
y_pred_cv = cross_val_predict(dt, X, y, cv=5)
mse = mean_squared_error(y, y_pred_cv)
print("📉 Cross-Validated MSE:", mse)